In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

## 1. read data from cities

In [2]:
output_csv_path="../WeatherPy/data/cities.csv"
cities_data=pd.read_csv(output_csv_path)
cities_data.dropna(inplace = True)
cities_data.head()

,Unnamed: 0,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.19,168.86,62.01,62.01,63,100,5.99,NZ,1603529781
1,1,carnarvon,-24.87,113.63,78.80,78.80,65,0,19.46,AU,1603529782
2,2,rikitea,-23.12,-134.97,74.10,74.10,83,99,11.68,PF,1603529665
3,3,tuatapere,-46.13,167.68,62.01,62.01,63,100,5.99,NZ,1603529783
4,4,coalinga,36.14,-120.36,56.70,60.01,44,25,2.15,US,1603529784


## build data and config Heatmap

In [3]:
# locations in a value
locations= cities_data[["Lat","Lng"]]

humid=cities_data.iloc[:,6]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(map_type="TERRAIN")

fig = gmaps.figure(center=(0, 0), zoom_level=2.25)

heat_layer= gmaps.heatmap_layer(locations,weights=humid.values,
                                dissipating=False,max_intensity=100,
                                point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## build a dataFrame filter

In [4]:
filter_data=cities_data.loc[((cities_data["Temp"]<80)
                 & (cities_data["Temp"]>70)
                 & (cities_data["Wind Speed"]<10.00)
                 & (cities_data["Cloudiness"]==0)
                )]
filter_data

,Unnamed: 0,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,bourail,-21.57,165.48,76.53,76.53,73,0,5.26,NC,1603529798
104,104,faya,18.39,42.45,72.50,73.40,15,0,8.05,SA,1603529853
201,201,jadu,31.95,12.03,74.73,74.73,22,0,3.47,LY,1603529935
224,224,awjilah,29.11,21.29,74.66,74.66,49,0,9.89,LY,1603529952
226,226,oussouye,12.48,-16.55,76.12,77.00,94,0,6.15,SN,1603529953
270,270,abha,18.22,42.51,72.45,73.40,15,0,8.05,SA,1603529984
380,380,cayenne,4.93,-52.33,75.20,75.20,100,0,1.12,GF,1603530092
422,422,derbent,42.07,48.29,70.03,70.03,50,0,7.92,RU,1603530121
451,451,rio branco,-9.97,-67.81,75.20,75.20,100,0,3.36,BR,1603530002
541,541,kharan,28.58,65.42,73.94,73.94,19,0,2.55,PK,1603530073


### build filter data to a hotal data 
1. add data column to DataFrame
2. find data with 5000 meters
3. using google maps search result
4. store the firset data result into DataFrame
5. set markers on the top of heatmap

In [5]:
hotel_df = filter_data[["City", "Lat", "Lng", "Humidity", "Wind Speed", "Country"]]
hotel_df=hotel_df.copy()
hotel_df.loc[:,"Hotel"]=None
hotel_df

,City,Lat,Lng,Humidity,Wind Speed,Country,Hotel
26,bourail,-21.57,165.48,73,5.26,NC,None
104,faya,18.39,42.45,15,8.05,SA,None
201,jadu,31.95,12.03,22,3.47,LY,None
224,awjilah,29.11,21.29,49,9.89,LY,None
226,oussouye,12.48,-16.55,94,6.15,SN,None
270,abha,18.22,42.51,15,8.05,SA,None
380,cayenne,4.93,-52.33,100,1.12,GF,None
422,derbent,42.07,48.29,50,7.92,RU,None
451,rio branco,-9.97,-67.81,100,3.36,BR,None
541,kharan,28.58,65.42,19,2.55,PK,None


In [6]:
google_map_api="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}
for index, row  in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"] =f"{lat},{lng}"
    hotel_name = requests.get(google_map_api, params=params).json()
    result=hotel_name["results"]
    if len(result)>0 :
        hotel_df.loc[index,"hotal"] =result[0]["name"]
    else:
        print(f"City {index} was not found hotal")


City 104 was not found hotal
City 201 was not found hotal
City 224 was not found hotal
City 226 was not found hotal


In [7]:
hotel_df.sort_values(["City"], inplace=True)
hotel_df.reset_index(inplace =True)
hotel_df

,index,City,Lat,Lng,Humidity,Wind Speed,Country,Hotel,hotal
0,270,abha,18.22,42.51,15,8.05,SA,None,Abha Palace Hotel
1,224,awjilah,29.11,21.29,49,9.89,LY,None,NaN
2,26,bourail,-21.57,165.48,73,5.26,NC,None,Betikure Parc Lodge
3,380,cayenne,4.93,-52.33,100,1.12,GF,None,Royal Amazonia
4,660,cosala,24.41,-106.69,93,3.27,MX,None,Cabañas El Potrero
5,422,derbent,42.07,48.29,50,7.92,RU,None,Arbat
6,617,dharchula,29.85,80.53,22,6.62,IN,None,Hotel new yash dharchula
7,104,faya,18.39,42.45,15,8.05,SA,None,NaN
8,201,jadu,31.95,12.03,22,3.47,LY,None,NaN
9,541,kharan,28.58,65.42,19,2.55,PK,None,Mansoor Hotel & Restaurant


### using template to create point


In [8]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [9]:
fig = gmaps.figure(map_type="TERRAIN")
fig = gmaps.figure(center=(0, 0), zoom_level=2.25)

heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=100, point_radius=1)
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# display
fig

Figure(layout=FigureLayout(height='420px'))